In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils import data
from PIL import Image
import numpy as np
import os, random
import json
import signal


In [3]:
data_dir = 'drive/MyDrive/Data'
train_dir = data_dir+'/training'
valid_dir = data_dir+'/validation'
test_dir = data_dir+'/testing'

In [4]:
train_transform = transforms.Compose(
    [transforms.RandomRotation(30),
     transforms.RandomResizedCrop(224),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
)

test_valid_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
train_data = ImageFolder(train_dir,train_transform)
valid_data = ImageFolder(valid_dir,test_valid_transform)
test_data = ImageFolder(test_dir,test_valid_transform)

In [23]:
print("Class labels:", train_data.classes)
print("Class labels:", test_data.classes)
print("Class labels:", valid_data.classes)

Class labels: ['0.1', '0.2', '0.5', '1', '10', '2', '5']
Class labels: ['0.1', '0.2', '0.5', '1', '10', '2', '5']
Class labels: ['0.1', '0.2', '0.5', '1', '10', '2', '5']


In [7]:
train_data_loader = data.DataLoader(train_data, batch_size=64, shuffle=True)
valid_data_loader = data.DataLoader(valid_data, batch_size=64)
test_data_loader = data.DataLoader(test_data, batch_size=64)

In [8]:
model = torchvision.models.vgg16(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

in_features_of_pretrained_model = model.classifier[0].in_features
classifier_out_features = len(train_data.classes)

classifier = nn.Sequential(nn.Linear(in_features=in_features_of_pretrained_model, out_features=2048, bias=True),
                           nn.ReLU(inplace=True),
                           nn.Dropout(p=0.2),
                           nn.Linear(in_features=2048, out_features=classifier_out_features, bias=True),
                           nn.LogSoftmax(dim=1)
                          )

model.classifier = classifier

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 106MB/s]


In [11]:
criterion = nn.NLLLoss()

# specify optimizer, using only classifer params. don't want to change the rest of VGG19
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

# move device to cpu, before moving it to cuda/gpu
device = 'cuda'
model.to(device)

# init variables for tracking loss/steps etc.
epochs = 5
for e in range(epochs):
  step = 0
  running_train_loss = 0
  running_valid_loss = 0

  # for each batch of images
  for images, labels in train_data_loader:
    step += 1
     # turn model to train mode
    model.train()
    # move images and model to device
    images, labels = images.to(device), labels.to(device)
    # zeroise grad
    optimizer.zero_grad()
  # forward
    outputs = model(images)
    train_loss = criterion(outputs, labels)
    train_loss.backward()
    optimizer.step()
    running_train_loss += train_loss.item()
    if step % 20 == 0 or step == 1 or step == len(train_data_loader):
      print("Epoch: {}/{} Batch % Complete: {:.2f}%".format(e+1, epochs, (step)*100/len(train_data_loader)))

  model.eval()
  with torch.no_grad():
    running_accuracy = 0
    running_valid_loss = 0
    for images, labels in valid_data_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      valid_loss = criterion(outputs, labels)
      running_valid_loss += valid_loss.item()
      ps = torch.exp(outputs)
      top_p, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      running_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        # print stats
  average_train_loss = running_train_loss/len(train_data_loader)
  average_valid_loss = running_valid_loss/len(valid_data_loader)
  accuracy = running_accuracy/len(valid_data_loader)
  print("Train Loss: {:.3f}".format(average_train_loss))
  print("Valid Loss: {:.3f}".format(average_valid_loss))
  print("Accuracy: {:.3f}%".format(accuracy*100))



Epoch: 1/5 Batch % Complete: 7.14%
Epoch: 1/5 Batch % Complete: 100.00%
Train Loss: 13.794
Valid Loss: 1.553
Accuracy: 42.566%
Epoch: 2/5 Batch % Complete: 7.14%
Epoch: 2/5 Batch % Complete: 100.00%
Train Loss: 1.397
Valid Loss: 1.093
Accuracy: 59.848%
Epoch: 3/5 Batch % Complete: 7.14%
Epoch: 3/5 Batch % Complete: 100.00%
Train Loss: 1.066
Valid Loss: 0.869
Accuracy: 65.483%
Epoch: 4/5 Batch % Complete: 7.14%
Epoch: 4/5 Batch % Complete: 100.00%
Train Loss: 0.798
Valid Loss: 0.804
Accuracy: 69.981%
Epoch: 5/5 Batch % Complete: 7.14%
Epoch: 5/5 Batch % Complete: 100.00%
Train Loss: 0.678
Valid Loss: 0.669
Accuracy: 71.970%


In [12]:
model.class_to_idx = train_data.class_to_idx
checkpoint = {'classifier': model.classifier,
              'state_dict': model.state_dict(),
              'epochs': epochs,
              'optim_stat_dict': optimizer.state_dict()
             }

torch.save(checkpoint, 'drive/MyDrive/checkpoint.pth')

In [13]:
model.eval()
with torch.no_grad():
    # for each batch of images
    running_accuracy = 0
    running_loss = 0
    for images, labels in test_data_loader:

        # move images and model to device
        images, labels = images.to(device), labels.to(device)

        # forward
        outputs = model(images)

        # loss
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # accuracy
        ps = torch.exp(outputs)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

average_loss = running_loss/len(valid_data_loader)
total_accuracy = accuracy/len(valid_data_loader)
print("Test Loss: {:.3f}".format(average_loss))
print("Test Accuracy: {:.3f}".format(total_accuracy*100))

Test Loss: 0.861
Test Accuracy: 90.077


In [15]:
def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    pil_image = Image.open(image_path)


    # TODO: Process a PIL image for use in a PyTorch model

    # reize
    pil_image.resize((256,256))

    # centre crop
    width, height = pil_image.size   # Get dimensions
    new_width, new_height = 224, 224

    left = round((width - new_width)/2)
    top = round((height - new_height)/2)
    x_right = round(width - new_width) - left
    x_bottom = round(height - new_height) - top
    right = width - x_right
    bottom = height - x_bottom

    # Crop the center of the image
    pil_image = pil_image.crop((left, top, right, bottom))

    # convert colour channel from 0-255, to 0-1
    np_image = np.array(pil_image)/255

    # normalize for model
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean)/std

    # tranpose color channge to 1st dim
    np_image = np_image.transpose((2 , 0, 1))

    # convert to Float Tensor
    tensor = torch.from_numpy(np_image)
    tensor = tensor.type(torch.FloatTensor)

    # return tensor
    return tensor


In [20]:
def predict(image_path, model, topk=5, device="cuda"):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    image = process_image(image_path)
    image = image.unsqueeze(0)

    # move to device
    image = image.to(device)
    model.eval()
    with torch.no_grad():
        ps = torch.exp(model(image))

    top_ps, top_classes = ps.topk(topk, dim=1)

    return top_ps.tolist()[0], top_classes.tolist()[0]


In [40]:
value = predict('/content/drive/MyDrive/images_test/1dh.jpeg',model)

In [41]:
print(value)

([0.34837090969085693, 0.20030103623867035, 0.1602628231048584, 0.14086829125881195, 0.058729853481054306], [2, 3, 0, 1, 6])


In [31]:
print("Class labels:", train_data.classes)

Class labels: ['0.1', '0.2', '0.5', '1', '10', '2', '5']
